In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [ ]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)  

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

download_file_from_google_drive('1b-gdpO9D44BPMDPE4FpC0FYPgqCmCskz', 'export1234.pkl')
# MODEL_URL = "https://drive.google.com/uc?export=download&id=1b-gdpO9D44BPMDPE4FpC0FYPgqCmCskz"
# urllib.request.urlretrieve(MODEL_URL, "export123.pkl")

path = Path()
learn_inf = load_learner(path/'export1234.pkl')

In [ ]:
# path = Path()
# learn_inf = load_learner('https://drive.google.com/file/d/1b-gdpO9D44BPMDPE4FpC0FYPgqCmCskz/view?usp=sharing',cpu=True)

In [ ]:
btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    #out_pl = widgets.Output()
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.02f}'
    display(lbl_pred)

btn_run.on_click(on_click_classify)

In [ ]:
VBox([widgets.Label('Upload a photo!'), 
      btn_upload, btn_run, out_pl, lbl_pred])